In [1]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision

In [3]:
# TODO

class RegressionModel(nn.Module):
    def __init__(self):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(28, 128) #input dim = 28, hidden = 300
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        

    def forward(self, x):
        a = F.relu(self.fc1(x))
        a = F.relu(self.fc2(a))
        a = F.relu(self.fc3(a))
        a = F.relu(self.fc4(a))
        
        return a

In [4]:
import set_data as sd

# load train set, test set

X_train, Y_train, X_test, Y_test = sd.set_data("training_data")

x_train = Variable(X_train)
y_train = Variable(Y_train)


In [5]:
# train
# hyperparameter

epochs = 5000
learningrate = 1e-3
# batch_size
# hidden_layer_size = 128

model = RegressionModel()
criterion = nn.MSELoss(size_average = True, reduction='mean') 
optimizer = torch.optim.Adam(model.parameters(), lr=learningrate)

print('\nStart Training...\n')

for epoch in range(epochs+1):
    # forward pass
    pred_y = model(x_train.float()) # 임의로 type convert 
    

    # compute and print loss
    loss = criterion(pred_y.double(), y_train) # 임의로 type convert

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    tmp_loss = []
    tmp_loss.append(loss.item())
    
    mbl = np.mean(np.sqrt(tmp_loss)).round(4)
    
    if epoch % 100 == 0:
        print ('epoch [{}/{}], loss {}'.format(epoch, epochs, mbl)) # or loss item
    

/nfs/home/kit7777777t/anaconda3/envs/LENNA/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))



Start Training...

epoch [0/5000], loss 230033.6895
epoch [100/5000], loss 229762.5752
epoch [200/5000], loss 223097.4322
epoch [300/5000], loss 186664.3298
epoch [400/5000], loss 109266.846
epoch [500/5000], loss 75781.9369
epoch [600/5000], loss 74239.0869
epoch [700/5000], loss 73813.5772
epoch [800/5000], loss 73457.8888
epoch [900/5000], loss 73120.2305
epoch [1000/5000], loss 72800.7779
epoch [1100/5000], loss 72498.8848
epoch [1200/5000], loss 72218.9104
epoch [1300/5000], loss 71958.7558
epoch [1400/5000], loss 71719.8149
epoch [1500/5000], loss 71496.039
epoch [1600/5000], loss 71287.2941
epoch [1700/5000], loss 71092.3028
epoch [1800/5000], loss 70898.7503
epoch [1900/5000], loss 70713.4005
epoch [2000/5000], loss 70537.8115
epoch [2100/5000], loss 70371.9144
epoch [2200/5000], loss 70212.8553
epoch [2300/5000], loss 70061.5787
epoch [2400/5000], loss 69912.8085
epoch [2500/5000], loss 69765.9314
epoch [2600/5000], loss 69623.252
epoch [2700/5000], loss 69482.9071
epoch [280

In [6]:
torch.save(model.state_dict(), "model.pt")

In [7]:
# test

print ('\nStart Testing...\n')

model.eval()

# calculate R squared
# TODO


Start Testing...



RegressionModel(
  (fc1): Linear(in_features=28, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=1, bias=True)
)

In [9]:
# visualize : TODO